In [60]:
import pandas as pd
from decouple import config
import os
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, Range1d
from bokeh.io import output_notebook
import json

output_notebook()
%matplotlib inline

Loading BokehJS ...

In [79]:
path = '/Users/vitaijdodonov/PycharmProjects/vhinny/vhinny-research/cache/feature_selection.json'
with open(path, 'r') as f:
    runs = json.load(f)
df_fi = pd.read_csv('/Users/vitaijdodonov/PycharmProjects/vhinny/vhinny-research/cache/feature_importance.csv')
df_fi = df_fi.set_index('feature')

In [80]:
d = {
    'run': [],
    'roc': [],
    'rem': [],
}

for run in runs:
    d['run'].append(int(run))
    d['roc'].append(runs[run].get('performance'))
    d['rem'].append(len(runs[run]['features_remaining']))
df = pd.DataFrame(d)

In [81]:
p = figure(
    title='Feature Selection',
    x_axis_label='Iteration',
    y_axis_label='ROC AUC',
    plot_width=700,
    plot_height=500
)

p.line(d['run'], d['roc'], line_width=2, line_color=(230, 54, 54), legend='ROC AUC Score')
p.y_range = Range1d(0.59, 0.83)

p.extra_y_ranges = {"Number of Features": Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name="Number of Features", axis_label="Number of Features"), 'right')
p.line(d['run'], d['rem'], line_width=2, line_color=(56, 56, 56), y_range_name='Number of Features', legend='Number of Features')

p.xaxis.axis_label_text_font_size = '16px'
p.yaxis.axis_label_text_font_size = '16px'
p.title.text_font_size = '20pt'
p.legend.label_text_font_size = '11pt'
p.legend.location = "bottom_left"
show(p)

In [82]:
df_sub = df_fi['class_5'].sort_values(ascending=False)[:25]
p = figure(plot_width=1000, plot_height=600, y_range=list(reversed(df_sub.reset_index()['feature'])))
p.hbar(y=df_sub.reset_index()['feature'], height=0.86, left=0, right=df_sub.reset_index()['class_5'], color=(230, 54, 54))
p.xaxis.axis_label_text_font_size = '16px'
p.yaxis.axis_label_text_font_size = '16px'
p.title.text_font_size = '20pt'

show(p)

In [83]:
df_100 = pd.read_csv(os.path.join(config('PYTHONPATH'), 'cache/class_5_lift_100.csv'))
df_9 = pd.read_csv(os.path.join(config('PYTHONPATH'), 'cache/class_5_lift.csv'))
centile = df_9['% of All Rows']
centile_100 = df_100['% of All Rows']
perfect_model = df_9['Perfect Rank Ordering']
random_model = df_9['Random Guess']
model = df_100['True Positive Rate (% of Positives Identified)']
model2 = df_9['True Positive Rate (% of Positives Identified)']

p = figure(
    title='Lift Curve',
    x_axis_label='Centile (% of All Rows)',
    y_axis_label='True Positive Rate (% of Positives Identified)',
    plot_width=600,
    plot_height=500
)
p.line(centile, perfect_model, line_width=2, line_color=(221, 221, 221), legend='Perfect Model')
p.line(centile, random_model, line_width=2, line_color=(69, 69, 69), legend='Random Model')
p.line(centile_100, model, line_width=0.78, line_color=(230, 54, 54), legend='Our Model V1 (100 features)', line_dash=[2])
p.line(centile, model2, line_width=2, line_color=(230, 54, 54), legend='Our Model V2 (9 features)')
p.xaxis.axis_label_text_font_size = '16px'
p.yaxis.axis_label_text_font_size = '16px'
p.title.text_font_size = '20pt'
p.legend.label_text_font_size = '11pt'
p.legend.location = "bottom_right"
show(p)

In [72]:
df_9

,Unnamed: 0,% of All Rows,True Positive Rate (% of Positives Identified),Perfect Rank Ordering,Random Guess
0,0,1.000000,1.000000,1.000000,1.000000
1,1,0.747185,0.971698,1.000000,0.747185
2,2,0.610031,0.933962,1.000000,0.610031
3,3,0.533265,0.924528,1.000000,0.533265
4,4,0.471853,0.915094,1.000000,0.471853
...,...,...,...,...,...
65,65,0.007165,0.047170,0.066038,0.007165
66,66,0.006141,0.037736,0.056604,0.006141
67,67,0.004094,0.037736,0.037736,0.004094
68,68,0.001024,0.009434,0.009434,0.001024
